<a href="https://colab.research.google.com/github/brenoskuk/recvis/blob/main/SinGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SinGan project

## MVA RecVis

### Collaborators: Breno Skuk and Claudia Torres Pérez

In [1]:
import numpy as np

In [2]:
!git clone https://github.com/tamarott/SinGAN.git

fatal: destination path 'SinGAN' already exists and is not an empty directory.


We need to change into the correct directory, then we install the required libraries

In [3]:
%cd SinGAN

/content/SinGAN


In [4]:
!python -m pip install -r requirements.txt

In [5]:
!python main_train.py --input_name mountains3.png

Random Seed:  9852
trained model already exist


In [9]:
!python random_samples.py --input_name mountains3.png --mode random_samples --gen_start_scale 0

Random Seed:  9174
Traceback (most recent call last):
  File "random_samples.py", line 40, in <module>
    Gs, Zs, reals, NoiseAmp = functions.load_trained_pyramid(opt)
  File "/content/SinGAN/SinGAN/functions.py", line 238, in load_trained_pyramid
    Gs = torch.load('%s/Gs.pth' % dir)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'TrainedModels/mountains3/scale_factor=0.750000,alpha=10/Gs.pth'


In [10]:
!ls SinGAN

functions.py  manipulate.py  __pycache__  training.py
imresize.py   models.py      SinGAN
